# Visualization Code for OddLot

This Jupyter Notebook will hold all of the code for creating the visualizations used in Charon SSG's slide deck, as well as explanations for what the visualizations mean. There will also be code for the calculation of statistically important financial metrics, such as the Sharpe ratio.

***Visualizations:***
1. Time plot with lines for total tender offers and odd lot tender offers
3. Charts visualizing best case and worst case returns (annualized and absolute returns)
2. Charts visualizing expected annualized return for each year and expected absolute return over time 
4. Charts visualizing expected annualized return correlated with quantitative variables
5. Distribution of type of tender offer
6. A chart that demonstrates if there is an association between expected annualized return and tender type
7. Visualization of return related to stock price

***Calculations:***
1. Calculate Sharpe ratio and Sortino ratio
2. Calculate outliers

## Base Code
This base code will be needed for all of the visualizations. So I'm putting it right at the start. This includes things such as importing needed modules and reading in and cleaning the data.

Estimated return is simply shares buying back/outstanding shares, discretized by the increments


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime

# Constants
####
# TODO: get montly treasury rate and average that across year
RISK_FREE_RATE = .0469 # MEAN 20 YEAR TREASURY RATE
####

odd_lot_data = pd.read_csv('./data/cleaned-data/url_data.csv')

# Remove any data rows where there have been errors or duplicates
odd_lot_data.dropna(subset=['exp date', 'trading px'], inplace=True)
odd_lot_data.drop_duplicates(subset=['cik', 'date'], inplace = True)

# Add average price
odd_lot_data['avg'] = (odd_lot_data['high'] - odd_lot_data['low']) / 2 + odd_lot_data['low']

# Make the date columns into actual python datetimes
odd_lot_data['date'] = pd.to_datetime(odd_lot_data['date'])
odd_lot_data['exp date'] = pd.to_datetime(odd_lot_data['exp date'])
odd_lot_data['px date'] = pd.to_datetime(odd_lot_data['px date'])
odd_lot_data['pay date'] = pd.to_datetime(odd_lot_data['exp date']).apply(lambda date: date +  datetime.timedelta(days=6))
# odd_lot_data['year'] = pd.DatetimeIndex(odd_lot_data['date']).year

odd_lot_data.sort_values('date', inplace=True)
odd_lot_data.reset_index(drop=True, inplace=True)

# Initialize fonts
georgia_font = {'fontname':'Georgia'}

## Flag Outliers

If an offer has crazy high return, I've flagged it so that we can take a look at it and make sure the values are correct.

In [2]:
# def calculate_sell_price(row):
#     if np.isnan(row['low']):
#         return row['tender px']
#     else:
#         return row['low']

# sell_prices = odd_lot_data.apply(lambda row: calculate_sell_price(row), axis=1)
# odd_lot_data['buy_low_px_spread'] = sell_prices - odd_lot_data['trading px']
# odd_lot_data['low_norm_return'] = odd_lot_data['buy_low_px_spread'] / odd_lot_data['trading px']

# highest_returns = odd_lot_data.sort_values(by='low_norm_return').tail(15)
# print(highest_returns)

## Simulation Variables
This section will handle any variables that we want to test, so that we can see what their effect is on returns.

In [3]:
### SET SIMULATOR VARIABLES
# tender_price_appreciation - how close doed a tender price gets to the low price
tender_price_appreciation = 0 # (increments: [0, .25, .5, .75, 1])
# Failure rates - what percent of tender offers do I have to sell back at market px
failure_rate = 0
# Fee rates - The fee rate for making a transaction

def get_appreciated_price(row):
    result = np.NaN
    if not np.isnan(row['low']):
        if row['low'] > row['trading px']:
            result = round(row['trading px'] + tender_price_appreciation * (row['low'] - row['trading px']), 2)
        else:
            result = row['trading px']
    else:
        if row['tender px'] > row['trading px']:
            result = round(row['trading px'] + tender_price_appreciation * (row['tender px'] - row['trading px']), 2)
        else:
            result = row['trading px']
    return result
#     print(row['low'])
#     print(row['tender px'])
#     print(row['trading px'])
#     print(result)
#     print('~~~~~~')
        
if tender_price_appreciation > 0:
    odd_lot_data['trading px'] = odd_lot_data.apply(lambda row: get_appreciated_price(row), axis=1)


## The Naive Strategy
Buy stocks of every tender lot offer, simple.

In [4]:
odd_lot_data['naive'] = 'y'
print(odd_lot_data['naive'].value_counts())

y    486
Name: naive, dtype: int64


## The Conservative Strategy
Only buy if the low price is above the initial buy price, ergo you can't ever lose money.

In [5]:
def conservative_strategy(row):
    if row['trading px'] <= row['low']:
        return 'y'
    elif row['trading px'] <= row['tender px']:
        return 'y'
    else:
        return 'n'
odd_lot_data['conservative'] = odd_lot_data.apply(lambda row: conservative_strategy(row), axis=1)
print(odd_lot_data['conservative'].value_counts())

y    286
n    200
Name: conservative, dtype: int64


## The Simulator
This code will simulate our return based on what instructions we give it.

In [6]:
# # Simulate trading of as many stocks as possible at a time
# def simulate_no_limit(principal, outlook, strategy):
#     portfolio = pd.DataFrame({'value': [principal], 'cash': [principal], 'date': [np.NaN]})
#     trade_dates = odd_lot_data['date'].append(odd_lot_data['exp date']).sort_values()
#     # Iterrate through every filing and pay day for the odd lot offers
#     for trade_date in trade_dates:
#         # Get any trades for that day
#         buy_trades = odd_lot_data[(odd_lot_data['date'] == trade_date) & (odd_lot_data[strategy] == 'y')]
#         sell_trades = odd_lot_data[(odd_lot_data['exp date'] == trade_date) & (odd_lot_data[strategy] == 'y')]
        
#         # Calculate the new portfolio value and cash after the day
#         new_port_value = portfolio.tail(1)['value'].values[0]
#         new_port_cash = portfolio.tail(1)['cash'].values[0]
#         for i, sell_trade in sell_trades.iterrows():
#             # Set sell price to whatever outlook we are expecting, if the value is nan, use the tender px
#             sell_price = sell_trade[outlook]
#             if np.isnan(sell_price) or sell_price is None:
#                 sell_price = sell_trade['tender px']
#             # Sell back the stocks
#             new_port_cash += 99 * sell_price
            
#             # Update portfolio value after full trade
#             new_port_value += 99 * (sell_price - sell_trade['trading px'])
#         for i, buy_trade in buy_trades.iterrows():
#             # In a buy trade, you buy as many stocks as possible
#             n_stocks = math.floor(new_port_cash / buy_trade['trading px'])
#             new_port_cash -= n_stocks * buy_trade['trading px']
        
#         portfolio = portfolio.append({'value': new_port_value, 
#                               'cash': new_port_cash, 
#                               'date': trade_date}, 
#                              ignore_index=True,)

#     return portfolio


# Simulate trading of 99 stocks at a time (99 stocks is the max for one account in odd lot)
def simulate_one_account_limit(principal, outlook, strategy):
    portfolio = pd.DataFrame({'value': [principal], 
                              'cash_invested': [0], 
                              'date': [pd.to_datetime('1/01/2000')]})
    trade_dates = odd_lot_data['date'].append(odd_lot_data['pay date']).sort_values().drop_duplicates()
    # Iterrate through every filing and pay day for the odd lot offers
    for trade_date in trade_dates:
#         print("\n\n\n\n" + str(trade_date))
        # Get any trades for that day
        buy_trades = odd_lot_data[(odd_lot_data['date'] == trade_date) & (odd_lot_data[strategy] == 'y')]
        sell_trades = odd_lot_data[(odd_lot_data['pay date'] == trade_date) & (odd_lot_data[strategy] == 'y')]
#         print(buy_trades)
#         print(sell_trades)
        
        # Calculate the new portfolio value and cash after the day
        new_port_value = portfolio.tail(1)['value'].values[0]
        cash_invested = portfolio.tail(1)['cash_invested'].values[0]
        for i, buy_trade in buy_trades.iterrows():
            # In a buy trade, you buy 99 stocks
            cash_invested += 99 * buy_trade['trading px']
        for i, sell_trade in sell_trades.iterrows():
            # Set sell price to whatever outlook we are expecting, if the value is nan, use the tender px
            sell_price = sell_trade[outlook]
            if np.isnan(sell_price) or sell_price is None:
                sell_price = sell_trade['tender px']
            
            # Sell back % of stocks at market price, i.e. the failure rate
            n_stocks = int(round(99 - 99 * failure_rate))
            print(n_stocks)
            
            # Sell back the stocks
            cash_invested -= 99 * sell_trade['trading px']
            # Update portfolio value after full trade
            new_port_value += n_stocks * (sell_price - sell_trade['trading px'])

        portfolio = portfolio.append({'value': new_port_value,
                                      'cash_invested': cash_invested,
                                      'date': trade_date}, 
                                     ignore_index=True,)
    return portfolio


def calculate_returns(portfolio, value_col):
    returns = np.empty(len(portfolio.index))
    for i in range(len(portfolio.index)):
        if i > 0:
            returns[i] = (portfolio.iloc[i][value_col] - portfolio.iloc[i-1][value_col])
        else:
            returns[i] = 0
    return returns

## Worst Case
Using outlook equals 'low', what is our return for the two strategies

In [7]:
portfolios = {}
# Calculate worst case for the naive strategy
low_naive_portfolio = simulate_one_account_limit(10000, 'low', 'naive')
# Calculate returns
low_naive_portfolio['return'] = calculate_returns(low_naive_portfolio, 'value')
low_naive_portfolio['norm return'] = low_naive_portfolio['return'] / low_naive_portfolio.iloc[0]['value']
# print(low_naive_portfolio)

# Calculate worst case for the conservative strategy
low_conservative_portfolio = simulate_one_account_limit(10000, 'low', 'conservative')
# Calculate returns
low_conservative_portfolio['return'] = calculate_returns(low_conservative_portfolio, 'value')
low_conservative_portfolio['norm return'] = low_conservative_portfolio['return'] / low_conservative_portfolio.iloc[0]['value']
# print(low_conservative_portfolio)

# Add the two worst cases to the portfolios array
portfolios['lnp'] = low_naive_portfolio
portfolios['lcp'] = low_conservative_portfolio

99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
9

## Average Case
Using outlook == 'avg', what is our return for the two strategies

In [8]:
# Calculate worst case for the naive strategy
avg_naive_portfolio = simulate_one_account_limit(10000, 'avg', 'naive')
# Calculate returns
avg_naive_portfolio['return'] = calculate_returns(avg_naive_portfolio, 'value')
avg_naive_portfolio['norm return'] = avg_naive_portfolio['return'] / avg_naive_portfolio.iloc[0]['value']
# print(avg_naive_portfolio)

# Calculate worst case for the conservative strategy
avg_conservative_portfolio = simulate_one_account_limit(10000, 'avg', 'conservative')
# Calculate returns
avg_conservative_portfolio['return'] = calculate_returns(avg_conservative_portfolio, 'value')
avg_conservative_portfolio['norm return'] = avg_conservative_portfolio['return'] / avg_conservative_portfolio.iloc[0]['value']
# print(avg_conservative_portfolio)

# Add the two best cases to the portfolios array
portfolios['anp'] = avg_naive_portfolio
portfolios['acp'] = avg_conservative_portfolio

99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
9

## Best Case
Using outlook equals 'high', what is our return for the two strategies

In [9]:
# Calculate worst case for the naive strategy
high_naive_portfolio = simulate_one_account_limit(10000, 'high', 'naive')
# Calculate returns
high_naive_portfolio['return'] = calculate_returns(high_naive_portfolio, 'value')
high_naive_portfolio['norm return'] = high_naive_portfolio['return'] / high_naive_portfolio.iloc[0]['value']
# print(high_naive_portfolio)

# Calculate worst case for the conservative strategy
high_conservative_portfolio = simulate_one_account_limit(10000, 'high', 'conservative')
# Calculate returns
high_conservative_portfolio['return'] = calculate_returns(high_conservative_portfolio, 'value')
high_conservative_portfolio['norm return'] = high_conservative_portfolio['return'] / high_conservative_portfolio.iloc[0]['value']
# print(high_conservative_portfolio)

# Add the two best cases to the portfolios array
portfolios['hnp'] = high_naive_portfolio
portfolios['hcp'] = high_conservative_portfolio

99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
9

## Absolute Returns
Get the absolute returns for the portfolios.


In [10]:
absolute_portfolios = None
for name, portfolio in portfolios.items():
    portfolio['year'] = pd.DatetimeIndex(portfolio['date']).year.astype(int)

    # Calculate annualized returns
    absolute_returns = []
    years = np.sort(portfolio['year'].unique())
    for year in years:
        year_data = portfolio[portfolio['year'] == year]
        start_value = year_data.iloc[0]['value']
        end_value = year_data.tail(1).iloc[0]['value']
        ####
        # TODO: divide by start value once we are full utilizing returns!!!
        ####
        absolute_returns.append(end_value - start_value)
        
    if absolute_portfolios is None:
        absolute_portfolios = pd.DataFrame({name: absolute_returns},
                                            index=years)
    else:
        absolute_portfolios[name] = absolute_returns
                
print(absolute_portfolios)
absolute_portfolios.to_csv('./data/results/failure_10_percent_absolute_portfolio_returns.csv')

            lnp        lcp         anp        acp         hnp         hcp
2000  3172.4940  3344.0415   4910.2190  4667.9165   6647.9440   5991.7915
2001  1403.2007  2766.6807   2237.5757  3189.4307   3071.9507   3612.1807
2002 -7825.3250  2425.2500  -3155.0500  2981.5000   1515.2250   3537.7500
2003  2658.5190  3210.3190   4187.9840  3740.7590   5717.4490   4271.1990
2004     5.8740  1383.9500   3067.4740  1953.5500   6129.0740   2523.1500
2005  1383.9500  3304.5700   5157.5500  4561.6950   8931.1500   5818.8200
2006  -848.1700  2185.8400   3968.9550  3120.3400   8786.0800   4054.8400
2007  4962.6400  7947.7000  10877.5800  9393.9500  16792.5200  10840.2000
2008  1394.6300  1639.3800   2507.1300  2235.6800   3619.6300   2831.9800
2009   209.1500   317.7300    471.7000   437.8800    734.2500    558.0300
2010  1329.6600  1741.7300   2357.1650  2386.9800   3384.6700   3032.2300
2011 -3792.7350   822.8050   1347.0150   967.4300   6486.7650   1112.0550
2012    63.1900  1250.4500   1954.8850

## Annualizing Data
We will now group the data up by year so that we can annualize it and analyze it better, starting in 2000.

In [11]:
annualized_portfolios = None
for name, portfolio in portfolios.items():
    portfolio['year'] = pd.DatetimeIndex(portfolio['date']).year.astype(int)

    # Calculate annualized returns
    annualized_returns = []
    years = np.sort(portfolio['year'].unique())
    for year in years:
        year_data = portfolio[portfolio['year'] == year]
        start_value = year_data.iloc[0]['value']
        end_value = year_data.tail(1).iloc[0]['value']
        ####
        # TODO: divide by start value once we are full utilizing returns!!!
        ####
        annualized_returns.append((end_value - start_value) / 10000)
        
    if annualized_portfolios is None:
        annualized_portfolios = pd.DataFrame({name: annualized_returns},
                                            index=years)
    else:
        annualized_portfolios[name] = annualized_returns
                
print(annualized_portfolios)
annualized_portfolios.to_csv('./data/results/failure_10_percent_annualized_portfolio_returns.csv')

           lnp       lcp       anp       acp       hnp       hcp
2000  0.317249  0.334404  0.491022  0.466792  0.664794  0.599179
2001  0.140320  0.276668  0.223758  0.318943  0.307195  0.361218
2002 -0.782532  0.242525 -0.315505  0.298150  0.151522  0.353775
2003  0.265852  0.321032  0.418798  0.374076  0.571745  0.427120
2004  0.000587  0.138395  0.306747  0.195355  0.612907  0.252315
2005  0.138395  0.330457  0.515755  0.456170  0.893115  0.581882
2006 -0.084817  0.218584  0.396895  0.312034  0.878608  0.405484
2007  0.496264  0.794770  1.087758  0.939395  1.679252  1.084020
2008  0.139463  0.163938  0.250713  0.223568  0.361963  0.283198
2009  0.020915  0.031773  0.047170  0.043788  0.073425  0.055803
2010  0.132966  0.174173  0.235717  0.238698  0.338467  0.303223
2011 -0.379274  0.082280  0.134701  0.096743  0.648677  0.111205
2012  0.006319  0.125045  0.195488  0.159043  0.384658  0.193041
2013 -0.045479  0.053845  0.102039  0.083437  0.249556  0.113030
2014  0.032574  0.093895 

## Analyzing portfolios
We will now calculate things like the standard deviation of returns, sharpe ratios, overall returns, etc.

In [12]:
analysis_data = pd.DataFrame(columns={'strategy', 'final_ret', 'ret_std', 'sharpe_ratio'})

# TODO: Use anualized returns/std deviation to calculate this stuff
for strategy in annualized_portfolios:
    portfolio = annualized_portfolios[strategy]
    
    row = {}
    row['strategy'] = strategy
    
    # Calculate cash needed to maintain a single account:
    
    # Calculate final return
    final_ret = 1
    for ret in portfolio.values:
        final_ret = final_ret * (1 + ret)
    row['final_ret'] = final_ret - 1
    
    # Calculate standard deviations
    row['ret_std'] = portfolio.std()
    
    # Calculate Sharpe Ratio using MEAN 20 YEAR TREASURY RATE as risk free rate
#     Sharpe ratio = (return - risk_free_return) / std_dev
    row['sharpe_ratio'] = (row['final_ret'] - .024) / row['ret_std']
    
    
    # Add to analysis dataframe
    analysis_data = analysis_data.append(row, ignore_index=True)

print(analysis_data)

  strategy  sharpe_ratio   ret_std    final_ret
0      lnp     -2.346333  0.265092    -0.597994
1      lcp    168.355976  0.174639    29.425503
2      anp    292.353473  0.289712    84.722337
3      acp    402.341270  0.219582    88.371083
4      hnp   8331.042539  0.418625  3487.603983
5      hcp    870.268040  0.274933   239.289408


## Getting Time Series Data
Just save the portfolio data

In [17]:
# for name, portfolio in portfolios.items():
#     time_series_data = portfolio.drop(['cash_invested', 'return','norm return'], axis=1).set_index('date')
#     time_series_data.to_csv('./data/results/time_series_data_' + name + '.csv')
    